In [33]:
import numpy as np
import os
import mldata
from DT_Model import ID3

In [34]:
file_path = '/home/decaf/Dropbox/EECS 440/dataset/example'

In [44]:
examples = np.array(mldata.parse_c45('volcanoes', file_path))
samples = examples[:, 1:-1].astype(float)
targets = np.array(list(map(lambda x: x == 'True', examples[:, -1])))
dt = ID3(10, False)

In [45]:
examples

array([['1', '1', '104.0', ..., '124.0', '96.0', 'True'],
       ['2', '1', '111.0', ..., '104.0', '99.0', 'True'],
       ['3', '1', '125.0', ..., '112.0', '119.0', 'True'],
       ...,
       ['2229', '16', '127.0', ..., '70.0', '82.0', 'False'],
       ['2230', '38', '128.0', ..., '123.0', '135.0', 'False'],
       ['2231', '12', '87.0', ..., '83.0', '75.0', 'False']], dtype='<U5')

In [46]:
labels = targets
attr = samples[:,-2]
print(labels)
print(attr)

[ True  True  True ... False False False]
[124. 104. 112. ...  70. 123.  83.]


# Debug Cont IG

In [49]:
og_ent = dt.entropy_of(labels)
cont_xy_pair = np.array([attr, labels]).T

In [50]:
sorted_xy_pair = cont_xy_pair[cont_xy_pair[:, 0].argsort(kind='mergesort')]
# list of the indexes of samples where class label changed
changed_idx = np.where(sorted_xy_pair[:, 1] != np.roll(sorted_xy_pair[:, 1], 1))[0]
print(changed_idx)

[   4    5    7    8   11   14   20   23   24   30   37   41   46   52
   57   65   80   82   95  101  116  127  137  150  163  168  181  203
  217  238  257  275  298  314  330  353  374  403  423  443  469  497
  521  549  565  593  617  640  665  690  717  749  772  792  823  841
  875  893  921  940  959  982 1013 1037 1059 1077 1107 1122 1152 1169
 1202 1218 1250 1265 1293 1305 1328 1344 1361 1375 1403 1417 1446 1454
 1477 1483 1497 1502 1533 1539 1564 1569 1593 1597 1626 1630 1655 1661
 1687 1690 1714 1715 1738 1741 1754 1756 1780 1785 1809 1811 1833 1839
 1858 1860 1873 1877 1896 1898 1918 1919 1960 1961 1987 1990 2006 2009
 2029 2031 2057 2058 2069 2070 2104 2105 2152 2153 2180 2181 2185 2186]


In [51]:
partition = (sorted_xy_pair[:,0][1013] + sorted_xy_pair[:,0][1013-1])/2

left_branch = sorted_xy_pair[sorted_xy_pair[:,0]<= partition]
right_branch = sorted_xy_pair[sorted_xy_pair[:,0] > partition]

In [52]:
p_left = (left_branch.shape[0])/float(sorted_xy_pair.shape[0])
p_right = (right_branch.shape[0])/float(sorted_xy_pair.shape[0])
print(p_left, p_right)

0.45405647691618106 0.5459435230838189


In [53]:
curr_ent = p_left * dt.entropy_of(left_branch[:,1]) + p_right * dt.entropy_of(right_branch[:,1])
print(og_ent - curr_ent)

0.05867824267574817


In [55]:
best_ig = -100.0
# traverse through all the indexes of the continious attributes where there's a change in the Class Label (The first element should not be considered)
for i in changed_idx[1:]:
    curr_partition = (sorted_xy_pair[:,0][i] + sorted_xy_pair[:,0][i-1])/2
    # partition list into 2 branches
    left_branch = sorted_xy_pair[sorted_xy_pair[:,0]<= curr_partition]
    right_branch = sorted_xy_pair[sorted_xy_pair[:,0] > curr_partition]
    p_left = (left_branch.shape[0])/float(sorted_xy_pair.shape[0])
    p_right = (right_branch.shape[0])/float(sorted_xy_pair.shape[0])

    curr_ent = p_left * dt.entropy_of(left_branch[:,1]) + p_right * dt.entropy_of(right_branch[:,1])
    curr_ig = og_ent - curr_ent

    # Finding the maximum information gain
    if best_ig < curr_ig:
        best_ig = curr_ig
        best_partition = curr_partition
best_ig, best_partition

(0.09112507464360031, 108.0)